In [40]:
from __future__ import absolute_import, division, print_function, unicode_literals

import datetime  # For datetime objects
from quantax.data_query_advance import local_get_stock_day_adv, local_get_index_day_adv

import numpy as np
import pandas as pd

# Import the backtrader platform
import backtrader as bt


In [2]:

def prepare_data(code):
    print('reading data... {}'.format(code))
    data = local_get_stock_day_adv(code, start=datetime.date(2010, 1, 1).strftime('%Y-%m-%d'),
                                   end=datetime.datetime.today().strftime('%Y-%m-%d')).to_qfq()
    formatted = data.data.reset_index().loc[:, ['date', 'open', 'high', 'low', 'close', 'volume']]
    formatted.rename(columns={'date': 'datetime'}, inplace=True)
    return formatted.set_index('datetime')


In [56]:
class BiasRatioCrossStrategy(bt.Strategy):
    params = (
        ('short', 20),
        ('mid', 60),
        ('long', 120)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add MovingAverageSimple indicator
        self.short = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.short)
        self.mid = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.mid)
        self.long = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.long)
        self.sar = bt.indicators.ParabolicSAR(self.datas[0], period=10, afmax=0.1, subplot=True)
        
        self.csr = (self.dataclose / self.short) - 1
        self.smr = (self.short / self.mid) - 1
        self.mlr = (self.mid / self.long) - 1

        # Indicators for the plotting show
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.ATR(self.datas[0], plot=False)
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('SAR, %.2f' % self.sar[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.csr > self.smr and self.smr > self.mlr and self.smr > self.smr[-1] and self.csr[-1] < self.smr[-1]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.csr < self.smr or self.csr < self.mlr or self.csr < 0:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [57]:
cerebro = bt.Cerebro()
cerebro.addstrategy(BiasRatioCrossStrategy)


0

In [58]:
stock_data = prepare_data('002414')
print('data samples:\n', stock_data.head())

data = bt.feeds.PandasData(dataname=stock_data,
                           fromdate=datetime.datetime(2019, 1, 1),
                           todate=datetime.datetime(2020, 11, 30))

cerebro.adddata(data)

reading data... 002414
data samples:
                 open      high       low     close    volume
datetime                                                    
2010-07-16  4.617814  4.820998  4.534693  4.704629  228739.0
2010-07-19  4.839469  5.079595  4.804374  4.990933  260221.0
2010-07-20  4.950297  5.236601  4.946602  5.123927  218849.0
2010-07-21  5.123927  5.162716  5.051889  5.090678  126467.0
2010-07-22  5.061124  5.135009  4.950297  5.070360  112106.0


In [59]:
cerebro.broker.setcash(100000)
cerebro.addsizer(bt.sizers.PercentSizer, percents=80)
cerebro.broker.setcommission(commission=0.0)
cerebro.addobserver(bt.observers.Value)
cerebro.addanalyzer(bt.analyzers.PositionsValue, _name='pos', cash=True)


In [60]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 100000.00
2019-07-02, SAR, 10.63
2019-07-03, SAR, 10.78
2019-07-04, SAR, 10.92
2019-07-05, SAR, 11.04
2019-07-08, SAR, 12.15
2019-07-09, SAR, 12.13
2019-07-10, SAR, 12.11
2019-07-11, SAR, 12.09
2019-07-12, SAR, 12.04
2019-07-15, SAR, 11.99
2019-07-16, SAR, 11.94
2019-07-17, SAR, 10.85
2019-07-18, SAR, 10.87
2019-07-19, SAR, 10.90
2019-07-22, SAR, 12.12
2019-07-23, SAR, 12.12
2019-07-24, SAR, 12.12
2019-07-25, SAR, 12.09
2019-07-26, SAR, 12.07
2019-07-29, SAR, 12.05
2019-07-30, SAR, 12.02
2019-07-31, SAR, 12.00
2019-08-01, SAR, 11.98
2019-08-02, SAR, 11.96
2019-08-05, SAR, 11.91
2019-08-06, SAR, 11.84
2019-08-07, SAR, 11.69
2019-08-08, SAR, 11.55
2019-08-09, SAR, 11.42
2019-08-12, SAR, 11.30
2019-08-13, SAR, 11.20
2019-08-14, SAR, 11.10
2019-08-15, SAR, 11.01
2019-08-16, SAR, 10.92
2019-08-19, SAR, 9.96
2019-08-20, SAR, 9.98
2019-08-21, SAR, 10.02
2019-08-22, SAR, 10.07
2019-08-23, SAR, 10.14
2019-08-26, SAR, 10.21
2019-08-27, SAR, 10.27
2019-08-28, SAR, 10.34


In [ ]:
cerebro.plot(iplot=False, style='candlestick')

In [30]:
values = results[0].stats.broker.value.get(size=len(results[0].stats.broker.value))
values_list = list(values)

In [48]:
pos = results[0].analyzers.getbyname('pos').get_analysis()
pos[datetime.date(2014, 1, 2)]

[0.0]

In [50]:
pos_dt = pd.DataFrame(pos).T
pos_dt.head()

,0
2014-01-02,0.0
2014-01-03,0.0
2014-01-06,0.0
2014-01-07,0.0
2014-01-08,0.0


In [51]:
pos_dt['values'] = values_list
pos_dt.head()

,0,values
2014-01-02,0.0,100000.0
2014-01-03,0.0,100000.0
2014-01-06,0.0,100000.0
2014-01-07,0.0,100000.0
2014-01-08,0.0,100000.0
